In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
import library as lib
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from gensim.models import Word2Vec
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 8] nodename nor servname provided, or not
[nltk_data]     known>


In [1]:
from gensim.models import Word2Vec

In [7]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
df = pd.read_pickle('preprocessed_data2.pkl')

In [34]:
df.head()

,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos,context_lemma_pos_tfidf,question_lemma_pos_tfidf
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.2), (beyonce, NN, 0.2), (start,..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(what, WP, 0.2), (area, NNS, 0.2), (beyonce, ..."
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.125), (beyonce, NN, 0.125), (le..."
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (city, NN, 0.2), (state, NN, 0..."
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (decade, NN, 0.2), (beyonce, N..."


In [35]:
type(df.context_lemma_pos_tfidf[0])

list

In [36]:
voc = lib.get_total_vocab(data=df)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
len(voc)

96682

In [38]:
glove_vectors = lib.get_glove_vectors(filepath='glove.6B/glove.6B.300d.txt',
                                     data=df, vocab=voc, 
                                     columns = ['context_lemma_pos', 'question_lemma_pos'])

In [39]:
glove_vectors['bee']

array([-2.6607e-01,  3.7760e-01, -3.7297e-01,  3.9632e-01, -2.5198e-01,
        2.3823e-01, -5.1209e-01,  1.1211e-01, -2.3773e-01, -3.7428e-01,
       -2.1506e-01, -3.7493e-01, -5.0230e-01,  1.2763e-01,  6.6347e-01,
        3.1148e-01,  9.7449e-02, -4.6298e-02, -2.1342e-01, -9.8251e-03,
        4.7481e-01, -2.5973e-01, -7.3478e-02,  1.8440e-01,  7.5455e-02,
       -6.2966e-02,  2.6432e-01, -5.7984e-01, -2.3107e-01,  2.1037e-01,
       -6.3662e-02, -1.4472e-01,  9.4807e-02,  2.3767e-02, -4.4796e-01,
        1.2828e-01,  2.6972e-01, -7.0229e-02,  3.1668e-01, -4.0475e-01,
       -5.6774e-02,  7.0430e-02,  7.4560e-01,  2.6541e-01,  2.7807e-03,
       -1.3609e-01,  1.0039e+00, -1.3536e-01,  1.1548e-01, -2.8822e-01,
        5.1668e-01,  4.1236e-02,  1.5980e-01,  1.9944e-01,  4.6439e-02,
        1.6194e-01,  1.1778e-01, -2.5286e-01,  3.1291e-01, -3.1275e-01,
        3.8737e-01,  6.3415e-02, -1.5682e-01,  4.3968e-01, -2.4797e-02,
       -2.8540e-01,  9.8685e-03,  7.3803e-01, -1.5455e-01,  6.48

In [45]:
x_df = df.drop(columns = ['answers', 'plausible_answers', 'answers_lemma_pos', 'plausible_answers_lemma_pos'])
y_df = df.drop(columns = x_df.columns)

In [46]:

x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(x_df,y_df, random_state=43, test_size=.2)

In [49]:
x_train_df.head()

,context,question,is_impossible,context_lemma_pos,question_lemma_pos,context_lemma_pos_tfidf,question_lemma_pos_tfidf
35207,"India started the construction of a 40,000-ton...",Who isn't constructing the Indian-made ship?,1,"[(india, NN), (start, VBD), (construction, NN)...","[(who, WP), (isnt, VBP), (construct, VBG), (in...","[(india, NN, 0.017543859649122806), (start, VB...","[(who, WP, 0.2), (isnt, VBP, 0.2), (construct,..."
16433,The investigation of the transmitted signals s...,Knowing the codes allowed Stanford researchers...,1,"[(the, DT), (investigation, NN), (transmit, VB...","[(know, VBG), (code, NNS), (allow, VBN), (stan...","[(the, DT, 0.0392156862745098), (investigation...","[(know, VBG, 0.1111111111111111), (code, NNS, ..."
22453,"Note: Besides ethnic groups, Slavs often ident...",Where are Vlaji located?,0,"[(note, NN), (besides, IN), (ethnic, JJ), (gro...","[(where, WRB), (vlaji, NN), (locate, VBD)]","[(note, NN, 0.008620689655172414), (besides, I...","[(where, WRB, 0.3333333333333333), (vlaji, NN,..."
63333,The ceremonial county of Somerset consists of ...,For how many years did Avon county exist?,1,"[(the, DT), (ceremonial, JJ), (county, NN), (s...","[(for, IN), (many, JJ), (year, NNS), (avon, VB...","[(the, DT, 0.05172413793103448), (ceremonial, ...","[(for, IN, 0.16666666666666666), (many, JJ, 0...."
104786,Initially the change in strategy caught the RA...,The bombing of the Thames Estuary cause how ma...,0,"[(initially, RB), (change, VBP), (strategy, NN...","[(the, DT), (bombing, NN), (thames, NNS), (est...","[(initially, RB, 0.011494252873563218), (chang...","[(the, DT, 0.125), (bombing, NN, 0.125), (tham..."


In [55]:
x_train_df.question_lemma_pos[22453]

[('where', 'WRB'), ('vlaji', 'NN'), ('locate', 'VBD')]